In [1]:
!pip install -Uqq ipdb
!pip install torchmetrics
!pip install torchsummary

In [2]:
import ipdb
import time

import numpy as np
import pandas as pd
import os
from os.path import join
import glob
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from PIL import Image
import random
from tqdm import tqdm


from collections import Counter, OrderedDict
import matplotlib.pyplot as plt

import torch
from torch import nn

from torchsummary import summary
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

from torchmetrics import Accuracy

from sklearn.preprocessing import LabelEncoder
import seaborn as sns

torch.manual_seed(123)
from tqdm.notebook import tqdm

In [3]:
## define parameters
train_on_gpu = torch.cuda.is_available() ## it will return True or False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)

CUDA is available!  Training on GPU ...
cuda:0


In [4]:
dataset_path = '/kaggle/input/places2-mit-dataset/train_256_places365standard/data_256/'

## Get the total No of Folders
main_folders = os.listdir( dataset_path )
len(main_folders)

24

In [5]:
dataset_path = '/kaggle/input/places2-mit-dataset/train_256_places365standard/data_256/'

## Get the total No of Folders
main_folders = os.listdir( dataset_path )
len(main_folders)

24

## Extracting the image names and class labels

In [6]:
list_global = []

for folder in sorted(main_folders): # a, b, c
    data_list = []
    sub_folders_1 = os.listdir( join( dataset_path, folder )  ) # a = 28
    print(f'No of sub-folders in folder {folder}: {len(sub_folders_1)}')
    
    for inner in sorted(sub_folders_1): # 28 folders in a
        
        sub_folders_2 = os.listdir( join( dataset_path, folder, inner ) )
        # print(inner, 'length:', len(sub_folders_2) )

        if len(sub_folders_2) < 100:
#             print()
#             print('*'*25)
#             print(f'{inner} has {len(sub_folders_2)} folders')
            
            
            for f in sub_folders_2:
                folder_inside_folder_ = os.listdir( join( dataset_path, folder, inner,f )  )
#                 print(f'.......{f} length: {len(folder_inside_folder_)}')
#                 print(folder_inside_folder_[:10])
                data_list.append( [ folder + '^' + inner + '^' + f + '#'+ image for image in  folder_inside_folder_] ) 
                # print('*'*25)
           
        else:
            folder_inside_folder = os.listdir( join( dataset_path, folder, inner )  )
#             print(f'.......{inner} length: {len(folder_inside_folder)}')
#             print(folder_inside_folder[:10])
            data_list.append(  [ folder + '^' + inner + '#'+ image for image in  folder_inside_folder ] ) 
    
    list_global.append( data_list )
#     print(f'\n########## Length of local list: {len(data_list)}')


No of sub-folders in folder a: 28
No of sub-folders in folder b: 42
No of sub-folders in folder c: 37
No of sub-folders in folder d: 15
No of sub-folders in folder e: 8
No of sub-folders in folder f: 17
No of sub-folders in folder g: 11
No of sub-folders in folder h: 15
No of sub-folders in folder i: 9
No of sub-folders in folder j: 5
No of sub-folders in folder k: 4
No of sub-folders in folder l: 15
No of sub-folders in folder m: 17
No of sub-folders in folder n: 3
No of sub-folders in folder o: 9
No of sub-folders in folder p: 23
No of sub-folders in folder r: 19
No of sub-folders in folder s: 30
No of sub-folders in folder t: 14
No of sub-folders in folder u: 2
No of sub-folders in folder v: 8
No of sub-folders in folder w: 10
No of sub-folders in folder y: 2
No of sub-folders in folder z: 1


## Create a dataframe with images and class labels

In [7]:
data_list = []

for folder in range( len(list_global) ): # 24
#     print(folder)
    
    for item in range( len(list_global[folder]) ): # 30 for first data only --> a 
        for i in range( len(list_global[folder][item]) ):
#             print( list_global[folder][item][i] )
            data_list.append( [list_global[folder][item][i]] )
    

In [8]:
# sorted(list_global[17][0])

In [9]:
# folder_inside_folder_

In [10]:
df = pd.DataFrame( data_list, columns = ['file_name'] )
df['class'] = df['file_name'].str.split('#').str[0]

print(df.shape)
df.head(5)

(1803460, 2)


,file_name,class
0,a^airfield#00002170.jpg,a^airfield
1,a^airfield#00004296.jpg,a^airfield
2,a^airfield#00002129.jpg,a^airfield
3,a^airfield#00001760.jpg,a^airfield
4,a^airfield#00001158.jpg,a^airfield


## write the csv 

In [12]:
df.to_csv('places_data.csv')